In [1]:
# LOAD Packages 
import uproot
import pandas as pd
import numpy as np
import awkward as ak
!pip install awkward_pandas
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
#matplotlib.rcParams.update(matplotlib.rcParamsDefault)
#matplotlib.rcParams['text.usetex'] = True
import matplotlib.pyplot as plt
import os
import math
import csv
import seaborn as sns
import sympy as smp
import sympy.stats
import json
from scipy import stats,signal,ndimage,special,fft
from scipy.optimize import curve_fit
from scipy.integrate import quad
from mpl_toolkits.axes_grid1 import make_axes_locatable

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Open CAF File 
showers = uproot.open("/global/cfs/cdirs/dune/users/ehinkle/nd_prototypes_ana/2x2_CC_pi0/caf_studies/plotting/first_pass_reco_CC1pi0_no_charged_mesons_selection.root")
showers["RecoBenchmarkTree"].show()
showers["RecoBenchmarkTree"].keys()
showers["RecoBenchmarkTree"][0].num_entries

# Load CAF File into DataFrame
ak_array = showers["RecoBenchmarkTree"].arrays(library="ak")
caf_df = ak.to_dataframe(ak_array)
caf_df = caf_df.reset_index(drop=True)
num_entries = len(caf_df)
print("Number of Entries: ", num_entries)
caf_df.head()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
reco_ixn_gamma_mult  | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
reco_ixn_e_mult      | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
reco_ixn_index       | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
reco_ixn_cont_gam... | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
reco_ixn_e_cont_mult | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
reco_ixn_muon_mult   | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
reco_ixn_chpi_mult   | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
reco_ixn_proton_mult | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
reco_ixn_chkaon_mult | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
reco_ixn_vtx_x_pos   | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
reco_ixn_vtx_y_pos   | std::

,reco_ixn_gamma_mult,reco_ixn_e_mult,reco_ixn_index,reco_ixn_cont_gamma_mult,reco_ixn_e_cont_mult,reco_ixn_muon_mult,reco_ixn_chpi_mult,reco_ixn_proton_mult,reco_ixn_chkaon_mult,reco_ixn_vtx_x_pos,...,true_ixn_vtx_z_pos,spill_index,file_index,event,run,subrun,caf_file_name,in_truth_dict,truth_dict_key,overlap
0,0,2,1.0,0,2,1,0,1,0,-32.811600,...,-86.0,91,0,87,0,0,MiniRun6.2_1E19_RHC.caf.0000000.CAF.flat.root,False,None,0.865608
1,2,0,0.0,0,0,1,0,1,0,52.986298,...,-1010.0,177,1,169,0,0,MiniRun6.2_1E19_RHC.caf.0000001.CAF.flat.root,False,None,0.978022
2,1,1,0.0,0,1,1,0,0,0,28.599300,...,-3016.0,67,2,65,0,0,MiniRun6.2_1E19_RHC.caf.0000002.CAF.flat.root,False,None,0.555556
3,2,0,0.0,1,0,1,0,0,0,-48.995701,...,-57.0,92,3,91,0,0,MiniRun6.2_1E19_RHC.caf.0000003.CAF.flat.root,True,3092-34000008,0.821608
4,2,0,1.0,0,0,1,0,0,0,7.759499,...,44.0,126,5,121,0,0,MiniRun6.2_1E19_RHC.caf.0000005.CAF.flat.root,False,None,0.978723


In [3]:
# How many interactions in truth? 
true_CC1pi0_no_charged_mesons = caf_df[caf_df["in_truth_dict"] == True]
true_CC1pi0_no_charged_mesons = true_CC1pi0_no_charged_mesons.reset_index(drop=True)
num_true_CC1pi0_no_charged_mesons = len(true_CC1pi0_no_charged_mesons)
print("Number of True CC1pi0 No Charged Mesons: ", num_true_CC1pi0_no_charged_mesons)
print("Total in selection: ", num_entries)


Number of True CC1pi0 No Charged Mesons:  579
Total in selection:  1871


In [4]:
# Load Truth/Background Dictionary
sig_bkg_file = open("/global/cfs/cdirs/dune/users/ehinkle/nd_prototypes_ana/2x2_CC_pi0/caf_studies/plotting/combined_CC1pi0_sig_bkg_dict.json")
sig_bkg_dict = json.load(sig_bkg_file)
sig_bkg_df = pd.DataFrame(sig_bkg_dict)
sig_bkg_df = sig_bkg_df.transpose()
sig_bkg_df["dict_key"] = sig_bkg_df.index
sig_bkg_df.head()

sig_bkg_signal = sig_bkg_df[sig_bkg_df["sig"] == True]
num_sig_true = len(sig_bkg_signal)
print("Number of True Signal: ", num_sig_true)
print("Decay Modes: ", np.unique(sig_bkg_signal["pi0_child_pdg"]))
cc1pi0_0chpi_signal_df = sig_bkg_signal[sig_bkg_signal["pi0_child_pdg"].apply(lambda x: np.array(x).shape == (1,2))]
print("Decay Modes: ", np.unique(cc1pi0_0chpi_signal_df["pi0_child_pdg"]))
num_sig_true_diphoton_decay = len(np.unique(cc1pi0_0chpi_signal_df["dict_key"]))
print("Number of CC 1pi0 0chpi Events: ", num_sig_true_diphoton_decay)

Number of True Signal:  1195
Decay Modes:  [list([[-11, 11, 22]]) list([[22, 22]])]
Decay Modes:  [list([[22, 22]])]
Number of CC 1pi0 0chpi Events:  1181


In [5]:
# Match across dataframes -- add truth info to caf_df
full_df = true_CC1pi0_no_charged_mesons
full_df = full_df.join(sig_bkg_df.set_index('dict_key'), on='truth_dict_key', how='left')
full_df.head()

,reco_ixn_gamma_mult,reco_ixn_e_mult,reco_ixn_index,reco_ixn_cont_gamma_mult,reco_ixn_e_cont_mult,reco_ixn_muon_mult,reco_ixn_chpi_mult,reco_ixn_proton_mult,reco_ixn_chkaon_mult,reco_ixn_vtx_x_pos,...,primary_shower_start_pxyz,primary_shower_conv_dist,nu_energy,q2,vtx_x,vtx_y,vtx_z,filepath,flow_event_id,file_number
0,2,0,0.0,1,0,1,0,0,0,-48.995701,...,[],[],5597.553223,1647888.75,-49.363853,-4.877169,-57.023895,/global/cfs/cdirs/dune/users/ehinkle/nd_protot...,91,3
1,1,1,1.0,1,1,1,0,0,0,-18.844501,...,[],[],8073.892578,510562.25,-18.892887,27.489594,-47.355831,/global/cfs/cdirs/dune/users/ehinkle/nd_protot...,49,6
2,1,1,1.0,0,0,1,0,1,0,37.688995,...,[],[],4635.06543,672824.25,37.460186,56.12896,-24.583715,/global/cfs/cdirs/dune/users/ehinkle/nd_protot...,122,6
3,2,0,1.0,1,0,1,0,0,0,-18.844501,...,"[[-0.24371077120304108, 1.8344693183898926, 1....",[40.71717512005239],4134.362793,477655.5,-17.418896,26.800625,25.118906,/global/cfs/cdirs/dune/users/ehinkle/nd_protot...,158,6
4,2,0,0.0,1,0,1,0,0,0,5.985901,...,"[[3.3098230361938477, 1.9948194026947021, -1.9...",[40.41684548600245],5761.463379,984005.4375,6.904,-51.905609,-58.808399,/global/cfs/cdirs/dune/users/ehinkle/nd_protot...,167,6


In [8]:
full_df_sig_count = full_df[full_df["sig"] == True]
full_df_sig_count = full_df_sig_count.reset_index(drop=True)
full_df_bkg_count = full_df[full_df["sig"] == False]
num_full_df_sig_count = len(full_df_sig_count)
num_full_df_bkg_count = len(full_df_bkg_count)
print("Number of True CC1pi0 No Charged Mesons Signal: ", num_full_df_sig_count)
print("Number of True CC1pi0 No Charged Mesons Background: ", num_full_df_bkg_count)

print("PURITY OF SELECTION: ", num_full_df_sig_count/num_entries)
print("EFFICIENCY OF SELECTION: ", num_full_df_sig_count/num_sig_true_diphoton_decay)

Number of True CC1pi0 No Charged Mesons Signal:  334
Number of True CC1pi0 No Charged Mesons Background:  245
PURITY OF SELECTION:  0.17851416354890434
EFFICIENCY OF SELECTION:  0.28281117696867064


In [7]:
# Set up df to look at containment by event: 
unique_events = np.unique(full_df_sig_count["truth_dict_key"])
containment_df = pd.DataFrame()
containment_df["truth_dict_key"] = unique_events

# Set allowances for xyz matching of shower starts
x_allowance = 1.5
y_allowance = 1.5
z_allowance = 1.5

# Loop over events to add info to check containment
for event in unique_events:

    # Initialize flags
    lead_match_found = False
    sublead_match_found = False
    current_lead_max_energy = 0
    current_sublead_max_energy = 0

    # Get all showers in event from full df
    event_df = full_df_sig_count[full_df_sig_count["truth_dict_key"] == event]
    event_df = event_df.reset_index(drop=True)

    # Initialize reco info
    containment_df.loc[containment_df["truth_dict_key"] == event, "lead_reco_E"] = 0
    containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_reco_E"] = 0
    containment_df.loc[containment_df["truth_dict_key"] == event, "num_reco_events"] = len(np.unique(event_df['flow_event_id'])) # number of reco events associated with this truth event
    containment_df.loc[containment_df["truth_dict_key"] == event, "num_extra_lead_reco_showers"] = 0
    containment_df.loc[containment_df["truth_dict_key"] == event, "num_extra_sublead_reco_showers"] = 0
    containment_df.loc[containment_df["truth_dict_key"] == event, "num_extra_unmatched_reco_showers"] = 0
    containment_df.loc[containment_df["truth_dict_key"] == event, "lead_reco_pdg"] = 0
    containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_reco_pdg"] = 0

    # Loop over showers in event to add to containment df
    num_reco_showers = len(event_df)

    for i in range(num_reco_showers):
        shower = event_df.iloc[i]

        # Some info only needs to be loaded once per event
        if i == 0:

            # Load true pi0 info 
            containment_df.loc[containment_df["truth_dict_key"] == event, "pi0_true_KE"] = shower["pi0_ke"][0]

            # Load available energy info
            if shower["pi0_child_available_energy"][0][0] > shower["pi0_child_available_energy"][0][1]:
                lead_shower_idx = 0
                sublead_shower_idx = 1
            else:
                lead_shower_idx = 1
                sublead_shower_idx = 0
            containment_df.loc[containment_df["truth_dict_key"] == event, "lead_available_E"] = shower["pi0_child_available_energy"][0][lead_shower_idx]
            containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_available_E"] = shower["pi0_child_available_energy"][0][sublead_shower_idx]

            # Load depo energy info
            containment_df.loc[containment_df["truth_dict_key"] == event, "lead_depo_E"] = shower["pi0_child_edep"][0][lead_shower_idx]
            containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_depo_E"] = shower["pi0_child_edep"][0][sublead_shower_idx]

            # Load primary shower info
            containment_df.loc[containment_df["truth_dict_key"] == event, "num_extra_true_primary_showers"] = len(shower["primary_shower_pdg"])

            # Load pi0 child deposition start information to match with reco showers
            lead_pi0_child_xyz = shower["pi0_child_edep_start"][0][lead_shower_idx]
            sublead_pi0_child_xyz= shower["pi0_child_edep_start"][0][sublead_shower_idx]
            containment_df.loc[containment_df["truth_dict_key"] == event, "lead_edep_start_x"] = lead_pi0_child_xyz[0]
            containment_df.loc[containment_df["truth_dict_key"] == event, "lead_edep_start_y"] = lead_pi0_child_xyz[1]
            containment_df.loc[containment_df["truth_dict_key"] == event, "lead_edep_start_z"] = lead_pi0_child_xyz[2]
            containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_edep_start_x"] = sublead_pi0_child_xyz[0]
            containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_edep_start_y"] = sublead_pi0_child_xyz[1]
            containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_edep_start_z"] = sublead_pi0_child_xyz[2]

        # Load shower info to check with truth info
        shower_start_x = shower["reco_shower_start_x"]
        shower_start_y = shower["reco_shower_start_y"]
        shower_start_z = shower["reco_shower_start_z"]

        # Check if reco shower matches with lead or sublead pi0 child based on start location of shower deposition
        if ((abs(lead_pi0_child_xyz[0] - shower_start_x) <= x_allowance) and \
            (abs(lead_pi0_child_xyz[1] - shower_start_y) <= y_allowance) and \
            (abs(lead_pi0_child_xyz[2] - shower_start_z) <= z_allowance)):

            if lead_match_found == True:
                containment_df.loc[containment_df["truth_dict_key"] == event, "num_extra_lead_reco_showers"] += 1
                if shower["reco_energy"] * 1000 > current_lead_max_energy:
                    current_lead_max_energy = shower["reco_energy"] * 1000 # convert to MeV
                    containment_df.loc[containment_df["truth_dict_key"] == event, "lead_reco_E"] = current_lead_max_energy
                    containment_df.loc[containment_df["truth_dict_key"] == event, "lead_reco_pdg"] = shower["reco_pdg"]
                continue
            if lead_match_found == False:
                lead_match_found = True
                current_lead_max_energy = shower["reco_energy"] * 1000 # convert to MeV
                containment_df.loc[containment_df["truth_dict_key"] == event, "lead_reco_E"] = current_lead_max_energy
                containment_df.loc[containment_df["truth_dict_key"] == event, "lead_reco_pdg"] = shower["reco_pdg"]
                continue

        if ((abs(sublead_pi0_child_xyz[0] - shower_start_x) <= x_allowance) and \
            (abs(sublead_pi0_child_xyz[1] - shower_start_y) <= y_allowance) and \
            (abs(sublead_pi0_child_xyz[2] - shower_start_z) <= z_allowance)):

            if sublead_match_found == True:
                containment_df.loc[containment_df["truth_dict_key"] == event, "num_extra_sublead_reco_showers"] += 1
                if shower["reco_energy"] * 1000 > current_sublead_max_energy:
                    current_sublead_max_energy = shower["reco_energy"] * 1000 # convert to MeV
                    containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_reco_E"] = current_sublead_max_energy
                    containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_reco_pdg"] = shower["reco_pdg"]
                continue
            if sublead_match_found == False:
                sublead_match_found = True
                current_sublead_max_energy = shower["reco_energy"] * 1000 # convert to MeV
                containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_reco_E"] = current_sublead_max_energy
                containment_df.loc[containment_df["truth_dict_key"] == event, "sublead_reco_pdg"] = shower["reco_pdg"]
                continue

        # Add to extra showers if not matched with lead or sublead
        containment_df.loc[containment_df["truth_dict_key"] == event, "num_extra_unmatched_reco_showers"] += 1


KeyError: 'reco_shower_start_x'

In [ ]:
# Add reco and depo efficiency columns
containment_df["lead_depo_efficiency"] = containment_df["lead_depo_E"] / containment_df["lead_available_E"]
containment_df["sublead_depo_efficiency"] = containment_df["sublead_depo_E"] / containment_df["sublead_available_E"]
containment_df["lead_reco_efficiency"] = containment_df["lead_reco_E"] / containment_df["lead_depo_E"]
containment_df["sublead_reco_efficiency"] = containment_df["sublead_reco_E"] / containment_df["sublead_depo_E"]
containment_df["lead_reco_efficiency"].fillna(0, inplace=True)
containment_df["sublead_reco_efficiency"].fillna(0, inplace=True)
